# Stock Analysis Tutorial

- Tutorial reference: https://excelsior-cjh.tistory.com/109
- Learning BeautifulSoup tutorial: https://www.youtube.com/watch?v=GjKQ6V_ViQE

### Import necessary libraries

In [13]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from urllib.parse import urlparse
from urllib.parse import parse_qs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import plotly.offline as offline
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Scrape the stock data from Korea Exchange (KRX)

- Webpage: https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage

In [3]:
# You should open 'stock_data.xls' file using pd.read_html() function
stock_data = pd.read_html('stock_data.xls', converters={'종목코드':str})[0]
# Get both stock name and code from data
stock_data = stock_data.iloc[:, :2]
stock_data = stock_data.rename(columns={'회사명':'Name', '종목코드': 'Code'})
stock_data.head()

,Name,Code
0,상상인제3호스팩,415580
1,신영스팩7호,419270
2,포바이포,389140
3,신한제9호스팩,405640
4,미래에셋비전스팩1호,412930


### Scrape the sise table (daily stock price) from NAVER Finance

- Webpage: https://finance.naver.com/

In [21]:
# Define function that returns stock page url
def find_stock_page_url(stock_name, stock_data):
    stock_code = stock_data.loc[stock_data['Name'] == f'{stock_name}']['Code'].to_string(index=False)
    url = f'https://finance.naver.com/item/sise.naver?code={stock_code}'
    return url

stock_name = '카카오'
naver_finance_url = 'https://finance.naver.com'
stock_page_url = find_stock_page_url(stock_name, stock_data)

response = requests.get(stock_page_url)
# Convert to a BeautifulSoup object
stock_page = bs(response.content, 'html.parser')
iframe_src = stock_page.select('iframe[title="일별 시세"]')[0].attrs['src']
iframe_src_url = naver_finance_url + iframe_src

# Using selenium 
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(iframe_src_url)
iframe_src_html = driver.page_source

sise_df = pd.DataFrame()
sise_page = bs(iframe_src_html)

# Method 1 (preferred)
# Change page number to get the data from the first page to the page you wanted
page_num = 15

# If there is not enough data to scrape (means last page number < 15), get the stock data from first page to the last page
last_page_url = sise_page.select('table.Nnavi a')[-1]['href']
parsed_url = urlparse(last_page_url)
last_page_num = int(parse_qs(parsed_url.query)['page'][0])
if page_num < last_page_num:
    for num in range(1, page_num + 1):
        sise_page_url = iframe_src_url + f'&page={num}'
        driver.get(sise_page_url)
        sise_page_html = driver.page_source
        sise_page_table = pd.read_html(sise_page_html, header=0)[0]
        sise_page_table.dropna(how='any', inplace=True)
        sise_df = sise_df.append(sise_page_table)
else:
    for num in range(1, last_page_num + 1):
        sise_page_url = iframe_src_url + f'&page={num}'
        driver.get(sise_page_url)
        sise_page_html = driver.page_source
        sise_page_table = pd.read_html(sise_page_html, header=0)[0]
        sise_page_table.dropna(how='any', inplace=True)
        sise_df = sise_df.append(sise_page_table)

# Method 2 (restricted)
# Get the data only from the first page to page 11 (scrape table.Nnavi using select function from sise_page)

# page_list = sise_page.select('table.Nnavi a')[:-1]
# for page in page_list:
#     sise_page_url = naver_finance_url + str(page['href'])
#     driver.get(sise_page_url)
#     sise_page_html = driver.page_source
#     sise_page_table = pd.read_html(sise_page_html, header=0)[0]
#     sise_page_table.dropna(how='any', inplace=True)
#     sise_df = sise_df.append(sise_page_table)

sise_df = sise_df.rename(columns={'날짜':'Date', '종가':'Closing Price', '전일비':'Change', '시가':'Opening Price', '고가':'Highest', '저가':'Lowest', '거래량':'Volume'})
sise_df[['Closing Price', 'Change', 'Opening Price', 'Highest', 'Lowest', 'Volume']] = sise_df[['Closing Price', 'Change', 'Opening Price', 'Highest', 'Lowest', 'Volume']].apply(pd.to_numeric)
sise_df['Date'] = pd.to_datetime(sise_df['Date'])
sise_df = sise_df.sort_values(by='Date', ascending=True, ignore_index=True)
sise_df



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Ji Hun\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


,Date,Closing Price,Change,Opening Price,Highest,Lowest,Volume
0,2021-10-08,117500.0,2000.0,121500.0,121500.0,117500.0,3930836.0
1,2021-10-12,113500.0,4000.0,117000.0,118500.0,113000.0,3810466.0
2,2021-10-13,117000.0,3500.0,114000.0,118500.0,114000.0,2731883.0
3,2021-10-14,121500.0,4500.0,118500.0,122000.0,118500.0,3891958.0
4,2021-10-15,122000.0,500.0,123000.0,123500.0,120000.0,2331425.0
...,...,...,...,...,...,...,...
145,2022-05-11,85400.0,1800.0,83800.0,85400.0,83300.0,1367542.0
146,2022-05-12,80700.0,4700.0,83700.0,84000.0,80600.0,2971934.0
147,2022-05-13,81800.0,1100.0,80700.0,81900.0,80300.0,1528530.0
148,2022-05-16,82300.0,500.0,83300.0,84300.0,82100.0,1598561.0


### Plot OHLC chart using plotly

- Plotly website: https://plotly.com/
- OHLC chart reference: https://stackoverflow.com/questions/64689342/plotly-how-to-add-volume-to-a-candlestick-chart

In [22]:
offline.init_notebook_mode(connected=True)

# Create subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=(stock_name, 'Volume'), row_width=[0.2, 0.7])

# Plot OHLC 
fig = fig.add_trace(go.Candlestick(x=sise_df['Date'], open=sise_df['Opening Price'], high=sise_df['Highest'], low=sise_df['Lowest'], close=sise_df['Closing Price'],  name='OHLC'), row=1, col=1)

# Bar trace for volumes 
fig = fig.add_trace(go.Bar(x=sise_df['Date'], y=sise_df['Volume'], showlegend=False), row=2, col=1)

# Do not show OHLC's rangeslider plot 
fig.update(layout_xaxis_rangeslider_visible=False)

# Hide weekends and holidays
# data from krx holidays: http://open.krx.co.kr/contents/MKD/01/0110/01100305/MKD01100305.jsp
xls_data = ['krx_holidays_data_2021.xls', 'krx_holidays_data_2022.xls']
df = pd.DataFrame()
for data in xls_data:
    holidays = pd.read_excel(data)
    df = df.append(holidays)
holidays_date = df.iloc[:, 0].to_list()
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"]), dict(values=holidays_date)])

fig.show()